In [ ]:
import pandas as pd
from pathlib import Path
import openpyxl as oxl
import numpy as np
import pkg_resources
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

In [ ]:
# --- Import des données ---
df = pd.read_excel(Path("data") / "credit.xlsx")
df.info()
# 1000 lignes, 21 colonnes
# 12 colonnes numériques (int64), 9 non numériques (object)

In [ ]:
# Encodage de la variable cible : 1 -> 0, 2 -> 1 puis afficher
df["Cible"] = df["Cible"].map({1: 0, 2: 1})
df["Cible"].value_counts()

In [ ]:
# --- Statistiques générales ---
ProfileReport(df, title="Rapport de données - Crédit")
# Pas de valeurs manquantes
# Garanties -> imbalanced
# Distributions semblent fortement assymétriques

# Corrélations fortes :
# entre Biens et Statut_domicile
# Montant_credit et Duree_credit

# A quoi correspondent les valeurs de :
# Historique_credit, Objet_credit, Situation_familiale, Garanties, Biens, Autres_credits, Statut_domicile, Type_emploi, Telephone

In [ ]:
# Sélection des variables numériques et variables catégorielles
# Cle à exclure (car identification de l'observation)
# Variables numériques : Age, Montant_credit, Duree_credit
# Variables catégorielles : toutes les autres
var_num = df[["Age", "Montant_credit", "Duree_credit"]]
var_cat = df.drop(columns=["Cle", "Age", "Montant_credit", "Duree_credit"])

In [ ]:
# --- Statistiques descriptives - Univariées - Variables numériques ---
print("Statistiques descriptives variables numériques :")
var_num.describe()

In [ ]:
# Distributions
for col in var_num.columns:
    plt.figure(figsize=(6, 4))
    sns.histplot(var_num[col], kde=True)
    plt.title(f"Distribution de {col}")
    plt.show()
# Asymétrie pour Age, Montant_credit, et distribution irrégulière et asymétrique pour Duree_credit

In [ ]:
# Valeur aberrantes
# Boxplot
for col in var_num.columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=var_num[col])
    plt.title(f"Boxplot de {col}")
    plt.show()
# 13 valeurs potentiellements atypiques dans Age
# Beaucoup dans Montant_credit
# 10 dans Duree_credit

In [ ]:
# --- Statistiques descriptives - Univariées - Variables catégorielles ---
print("Statistiques descriptives variables catégorielles :")
var_cat.describe()

In [ ]:
# --- Représentation variable Cible ---
prop = df["Cible"].value_counts(normalize=True) * 100
prop.plot(kind="bar", color=["skyblue", "salmon"])
plt.title("Proportion des classes de la variable Cible")
plt.xlabel("Cible")
plt.ylabel("Pourcentage (%)")
plt.show()

In [ ]:
# Barplot
for col in var_cat.columns:
    plt.figure(figsize=(8, 4))
    var_cat[col].value_counts().plot(kind="bar")
    plt.title(f"Répartition de {col}")
    plt.xlabel(col)
    plt.ylabel("Fréquence")
    plt.show()

In [ ]:
# Statistiques descriptives - Multivariées
# Par rapport à la variable Cible (en enlevant les variables continues numériques)
# Barplots croisés
for col in df.drop(
    columns=["Cible", "Cle", "Montant_credit", "Duree_credit", "Age"]
).columns:
    plt.figure(figsize=(6, 4))
    sns.countplot(x=col, hue="Cible", data=df)
    plt.title(f"{col} vs Cible")
    plt.show()

In [ ]:
# Histogrammes empilés
for col in df.drop(
    columns=["Cible", "Cle", "Montant_credit", "Duree_credit", "Age"]
).columns:
    cross_tab = pd.crosstab(df[col], df["Cible"], normalize="index")
    cross_tab.plot(kind="bar", stacked=True, figsize=(6, 4))
    plt.title(f"Proportion de Cible par {col}")
    plt.ylabel("Proportion")
    plt.show()

In [ ]:
# --- Corrélations ---
# Pearson
plt.figure(figsize=(10, 8))
corr_matrix = var_num.corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation - Variables numériques")
plt.show()
# Corrélation de 0.62 entre Montant_credit et Duree_credit

In [ ]:
# Khi2 et CramersV
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))

cat_cols = var_cat.columns
cramers = pd.DataFrame(
    np.zeros((len(cat_cols), len(cat_cols))), index=cat_cols, columns=cat_cols
)
for col1 in cat_cols:
    for col2 in cat_cols:
        cramers.loc[col1, col2] = cramers_v(var_cat[col1], var_cat[col2])
plt.figure(figsize=(10, 8))
sns.heatmap(cramers, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Corrélation Cramer's V - Variables catégorielles")
plt.show()
# Corrélation de 0.55 entre Statut_domicile et Biens